In [3]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Enhanced problem parameters
T = 5  # Number of periods (t=1,2,3,4,5)
I = ['Site_A', 'Site_B', 'Site_C', 'Site_D', 'Site_E']  # 5 sites
S = [f'Scenario_{i+1}' for i in range(10)]  # 10 scenarios

# Generate more realistic demand scenarios with different patterns
def generate_demand_scenarios():
    demand = {}
    base_demands = {'Site_A': 150, 'Site_B': 200, 'Site_C': 120, 'Site_D': 180, 'Site_E': 160}
    
    for s_idx, s in enumerate(S):
        demand[s] = {}
        # Create different scenario types: low, medium, high demand
        if s_idx < 3:  # Low demand scenarios
            multiplier = np.random.uniform(0.6, 0.8)
        elif s_idx < 7:  # Medium demand scenarios
            multiplier = np.random.uniform(0.9, 1.1)
        else:  # High demand scenarios
            multiplier = np.random.uniform(1.2, 1.5)
            
        for i in I:
            # Add some randomness around the base demand
            variation = np.random.uniform(0.8, 1.2)
            demand[s][i] = int(base_demands[i] * multiplier * variation)
    
    return demand

demand = generate_demand_scenarios()

# Equal probability for all scenarios
prob = {s: 1/len(S) for s in S}

# Enhanced parameters
Q = 250  # Capacity per warehouse unit
f = 800  # Fixed cost per warehouse unit
r0 = 1.5  # Stage 1 capacity cost per unit
R0 = 500  # Stage 1 max capacity
rt = {t: 2.0 + 0.2*t for t in range(1, T+1)}  # Stage 2 capacity cost (increasing)
Rt = {t: 150 for t in range(1, T+1)}  # Stage 2 max capacity per period

# Site-specific costs (different geographical locations)
ci = {'Site_A': 2.0, 'Site_B': 2.5, 'Site_C': 1.8, 'Site_D': 3.0, 'Site_E': 2.2}

# Time-varying unmet demand penalty
bt = {t: 8.0 + 1.0*t for t in range(1, T+1)}  # Increasing penalty over time
h = 0.15  # Holding cost per unit

# Create the Pyomo model
model = ConcreteModel()

# Sets
model.T = Set(initialize=range(1, T+1))
model.I = Set(initialize=I)
model.S = Set(initialize=S)

# Variables
# Stage 1 (t=0)
model.u0 = Var(within=NonNegativeReals)  # Initial capacity
model.y = Var(within=NonNegativeIntegers)  # Warehouse units rented

# Stage 2 (t >= 1, scenario-dependent)
model.u = Var(model.S, model.T, within=NonNegativeReals)  # Capacity added
model.x = Var(model.S, model.I, model.T, within=NonNegativeReals)  # Relief to sites
model.w = Var(model.S, model.T, within=NonNegativeReals)  # Unmet demand
model.z = Var(model.S, model.T, within=NonNegativeReals)  # Inventory

# Objective function
def total_cost_rule(model):
    # Stage 1 costs: fixed + capacity
    stage1_cost = f * model.y + r0 * model.u0

    # Stage 2 expected costs: capacity + unmet + holding + relief
    stage2_cost = sum(
        prob[s] * (
            sum(rt[t] * model.u[s, t] for t in model.T) + # Capacity expansion
            sum(bt[t] * model.w[s, t] for t in model.T) + # Unmet demand
            sum(h * model.z[s, t] for t in model.T) + # Holding
            sum(ci[i] * model.x[s, i, t] for i in model.I for t in model.T)
        )
        for s in model.S
    )
    return stage1_cost + stage2_cost

model.total_cost = Objective(rule=total_cost_rule, sense=minimize)




# Constraints

# Stage 1: Capacity limits
def stage1_capacity_rule(model):
    return model.u0 <= Q * model.y
model.stage1_capacity = Constraint(rule=stage1_capacity_rule)

def stage1_max_rule(model):
    return model.u0 <= R0
model.stage1_max = Constraint(rule=stage1_max_rule)



# Stage 2: Resource acquisition limits
def stage2_capacity_rule(model, s, t):
    return model.u[s, t] <= Rt[t]
model.stage2_capacity = Constraint(model.S, model.T, rule=stage2_capacity_rule)



# Demand satisfaction (total relief <= total demand)
def demand_satisfaction_rule(model, s, i):
    return sum(model.x[s, i, t] for t in model.T) == demand[s][i]
model.demand_satisfaction = Constraint(model.S, model.I, rule=demand_satisfaction_rule)



# Available capacity (relief <= depot capacity)
def capacity_balance_rule(model, s, t):
    if t == 1:
        return sum(model.x[s, i, 1] for i in model.I) <= model.u0 + model.u[s, 1]
    else:
        return sum(model.x[s, i, t] for i in model.I) <= model.z[s, t-1] + model.u[s, t]
model.capacity_balance = Constraint(model.S, model.T, rule=capacity_balance_rule)


def capacity_balance_rule_2(model, s, t):
    if t == 1:
        return sum(model.x[s, i, 1] for i in model.I) <= model.u0 + model.u[s, 1]
    else:
      return sum(model.x[s, i, t] for i in model.I) <= model.u0 + sum(model.u[s, k] for k in range(1,t)) - sum(model.x[s, i, k] for i in model.I for k in range(1,t-1))
model.capacity_balance_2 = Constraint(model.S, model.T, rule=capacity_balance_rule_2)




# Inventory balance
def inventory_rule(model, s, t):
    if t == 1:
        return model.z[s, 1] == model.u0 + model.u[s, 1] - sum(model.x[s, i, 1] for i in model.I)
    else:
        return model.z[s, t] == model.z[s, t-1] + model.u[s, t] - sum(model.x[s, i, t] for i in model.I)
model.inventory = Constraint(model.S, model.T, rule=inventory_rule)



# Unmet demand
def unmet_demand_rule(model, s, t):
    return model.w[s, t] == sum(demand[s][i] for i in model.I) - sum(model.x[s, i, k] for i in model.I for k in range(1, t))
model.unmet_demand = Constraint(model.S, model.T, rule=unmet_demand_rule)



# Solve the model
solver = SolverFactory('gurobi') 
results = solver.solve(model, tee=False)





# Check if solution is optimal
if results.solver.termination_condition == TerminationCondition.optimal:
    print("✓ Optimal solution found!")
else:
    print(f"⚠ Solver status: {results.solver.termination_condition}")



# RESULTS ANALYSIS

def create_results_tables():
    """Create results tables"""
    
    # Table 1: Stage 1 Decisions and Summary Statistics
    stage1_results = {
        'Decision Variable': ['Warehouse Units (y)', 'Initial Capacity (u₀)', 'Total Expected Cost'],
        'Optimal Value': [
            f"{int(model.y.value)}", 
            f"{model.u0.value:.2f}", 
            f"${model.total_cost():.2f}"
        ],
        'Unit': ['units', 'capacity units', 'USD']
    }
    
    # Table 2: Scenario-wise Total Demand
    scenario_demands = {}
    for s in S:
        total_demand = sum(demand[s][i] for i in I)
        scenario_demands[s] = total_demand
    
    demand_stats = {
        'Statistic': ['Mean', 'Std Dev', 'Min', 'Max', 'CV (%)'],
        'Total Demand': [
            f"{np.mean(list(scenario_demands.values())):.1f}",
            f"{np.std(list(scenario_demands.values())):.1f}",
            f"{min(scenario_demands.values())}",
            f"{max(scenario_demands.values())}",
            f"{100*np.std(list(scenario_demands.values()))/np.mean(list(scenario_demands.values())):.1f}"
        ]
    }
    
    # Table 3: Stage 2 Decisions Summary (averaged across scenarios)
    stage2_summary = []
    for t in model.T:
        avg_capacity = np.mean([model.u[s, t].value for s in S])
        avg_inventory = np.mean([model.z[s, t].value for s in S])
        avg_unmet = np.mean([model.w[s, t].value for s in S])
        total_relief = np.mean([sum(model.x[s, i, t].value for i in I) for s in S])
        
        stage2_summary.append({
            'Period': t,
            'Avg Capacity Added': f"{avg_capacity:.2f}",
            'Avg Total Relief': f"{total_relief:.2f}",
            'Avg Inventory': f"{avg_inventory:.2f}",
            'Avg Unmet Demand': f"{avg_unmet:.2f}"
        })
    
    # Table 4: Site-wise Relief Summary
    site_relief_summary = []
    for i in I:
        avg_relief = np.mean([sum(model.x[s, i, t].value for t in model.T) for s in S])
        total_demand = np.mean([demand[s][i] for s in S])
        service_rate = (avg_relief / total_demand) * 100 if total_demand > 0 else 0
        
        site_relief_summary.append({
            'Site': i,
            'Avg Total Demand': f"{total_demand:.1f}",
            'Avg Total Relief': f"{avg_relief:.1f}",
            'Service Rate (%)': f"{service_rate:.1f}",
            'Unit Cost': f"${ci[i]:.2f}"
        })
    
    return stage1_results, demand_stats, stage2_summary, site_relief_summary

def create_cost_breakdown():
    """Analyze cost components"""
    stage1_cost = f * model.y.value + r0 * model.u0.value
    
    # Calculate expected stage 2 costs
    capacity_cost = sum(prob[s] * sum(rt[t] * model.u[s, t].value for t in model.T) for s in S)
    unmet_cost = sum(prob[s] * sum(bt[t] * model.w[s, t].value for t in model.T) for s in S)
    holding_cost = sum(prob[s] * sum(h * model.z[s, t].value for t in model.T) for s in S)
    relief_cost = sum(prob[s] * sum(ci[i] * model.x[s, i, t].value for i in I for t in model.T) for s in S)
    
    total_cost = stage1_cost + capacity_cost + unmet_cost + holding_cost + relief_cost
    
    cost_breakdown = {
        'Cost Component': [
            'Stage 1 Fixed Costs', 
            'Stage 2 Capacity Expansion', 
            'Unmet Demand Penalties',
            'Inventory Holding Costs',
            'Relief Distribution Costs',
            'Total Expected Cost'
        ],
        'Amount (USD)': [
            f"{stage1_cost:.2f}",
            f"{capacity_cost:.2f}",
            f"{unmet_cost:.2f}",
            f"{holding_cost:.2f}",
            f"{relief_cost:.2f}",
            f"{total_cost:.2f}"
        ],
        'Percentage (%)': [
            f"{100*stage1_cost/total_cost:.1f}",
            f"{100*capacity_cost/total_cost:.1f}",
            f"{100*unmet_cost/total_cost:.1f}",
            f"{100*holding_cost/total_cost:.1f}",
            f"{100*relief_cost/total_cost:.1f}",
            "100.0"
        ]
    }
    
    return cost_breakdown

# Generate all results
stage1_results, demand_stats, stage2_summary, site_relief_summary = create_results_tables()
cost_breakdown = create_cost_breakdown()

# ============================================================================
# PRINT RESEARCH-QUALITY RESULTS
# ============================================================================

print("\n" + "="*80)
print("RESULTS FOR STOCHASTIC RELIEF DISTRIBUTION OPTIMIZATION")
print("="*80)

print("\nTable 1: Stage 1 Decisions and Model Summary")
print("-" * 50)
print(tabulate(stage1_results, headers='keys', tablefmt='grid'))

print("\nTable 2: Demand Scenario Statistics")
print("-" * 40)
print(tabulate(demand_stats, headers='keys', tablefmt='grid'))

print("\nTable 3: Expected Stage 2 Decisions by Period")
print("-" * 55)
print(tabulate(stage2_summary, headers='keys', tablefmt='grid'))

print("\nTable 4: Site-wise Relief Distribution Summary")
print("-" * 55)
print(tabulate(site_relief_summary, headers='keys', tablefmt='grid'))

print("\nTable 5: Cost Component Analysis")
print("-" * 45)
print(tabulate(cost_breakdown, headers='keys', tablefmt='grid'))

# Additional analysis
print("\n" + "="*80)
print("DETAILED SCENARIO ANALYSIS")
print("="*80)

# Scenario-wise performance metrics
scenario_analysis = []
for s in S:
    total_demand_s = sum(demand[s][i] for i in I)
    total_relief_s = sum(model.x[s, i, t].value for i in I for t in model.T)
    total_unmet_s = sum(model.w[s, t].value for t in model.T)
    service_level = (total_relief_s / total_demand_s) * 100 if total_demand_s > 0 else 0
    
    scenario_analysis.append({
        'Scenario': s,
        'Total Demand': f"{total_demand_s}",
        'Total Relief': f"{total_relief_s:.1f}",
        'Service Level (%)': f"{service_level:.1f}",
        'Total Unmet': f"{total_unmet_s:.1f}"
    })

print("\nTable 6: Scenario-wise Performance Analysis")
print("-" * 55)
print(tabulate(scenario_analysis, headers='keys', tablefmt='grid'))

# Key Performance Indicators
service_levels = [float(row['Service Level (%)']) for row in scenario_analysis]
print(f"\nKey Performance Indicators:")
print(f"• Average Service Level: {np.mean(service_levels):.1f}%")
print(f"• Service Level Std Dev: {np.std(service_levels):.1f}%")
print(f"• Minimum Service Level: {min(service_levels):.1f}%")
print(f"• Maximum Service Level: {max(service_levels):.1f}%")

print(f"\nCapacity Utilization:")
total_available = model.u0.value + sum(np.mean([model.u[s, t].value for s in S]) for t in model.T)
avg_used = np.mean([sum(model.x[s, i, t].value for i in I for t in model.T) for s in S])
print(f"• Total Available Capacity: {total_available:.1f}")
print(f"• Average Capacity Used: {avg_used:.1f}")
print(f"• Capacity Utilization Rate: {100*avg_used/total_available:.1f}%")

print("\n" + "="*80)
print("END OF ANALYSIS")
print("="*80)

✓ Optimal solution found!

RESULTS FOR STOCHASTIC RELIEF DISTRIBUTION OPTIMIZATION

Table 1: Stage 1 Decisions and Model Summary
--------------------------------------------------
+-----------------------+-----------------+----------------+
| Decision Variable     | Optimal Value   | Unit           |
+=======================+=================+================+
| Warehouse Units (y)   | 2               | units          |
+-----------------------+-----------------+----------------+
| Initial Capacity (u₀) | 500.00          | capacity units |
+-----------------------+-----------------+----------------+
| Total Expected Cost   | $15451.43       | USD            |
+-----------------------+-----------------+----------------+

Table 2: Demand Scenario Statistics
----------------------------------------
+-------------+----------------+
| Statistic   |   Total Demand |
+=============+================+
| Mean        |          796   |
+-------------+----------------+
| Std Dev     |          217